# BERT-Question and Answer

In [2]:
!pip3 install fse
!pip3 install transformers
!pip3 install torch
!git clone -l -s https://github.com/harpreetvirkk/NLP-QuestionAnswerSystem rep
%cd rep
!ls dataset

In [1]:
import math
import os
import re
import fse
import gensim
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd
from fse import SplitIndexedList
from fse.models import uSIF
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('stopwords')
import logging
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from nltk.tokenize import word_tokenize
from transformers import BertForQuestionAnswering, BertTokenizer
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sthor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sthor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\sthor\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
# Reading in the dataset
df1 = pd.read_csv('./dataset/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('./dataset/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('./dataset/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')
frames = [df1, df2, df3]
df = pd.concat(frames)

In [3]:
def getArticleText(file):
    fpath = './dataset/text_data/'+file+'.txt.clean'
    try:
        f = open(fpath, 'r')
        text = f.read()
    except UnicodeDecodeError:
        f = open(fpath, 'r', encoding = 'ISO-8859-1')
        text = f.read()
    return text

In [4]:
df = df.dropna(subset=['ArticleFile'])
df = df.dropna(subset=['Answer'])
df['ArticleText'] = df['ArticleFile'].apply(lambda x: getArticleText(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', x))
df = df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns')
df.head(2)

ArticleTitle  \
0  Abraham_Lincoln   
1  Abraham_Lincoln   

                                                            Question Answer  \
0  Was Abraham Lincoln the sixteenth President of the United States?  yes     
1  Was Abraham Lincoln the sixteenth President of the United States?  Yes.    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [5]:
df.columns

Index(['ArticleTitle', 'Question', 'Answer', 'ArticleText'], dtype='object')

In [6]:
# stop_words = set(stopwords.words('english'))
def cleanQuestion(text):
    text = str(text)
    # wnl = nltk.stem.WordNetLemmatizer()
    text = text.lower()
    words = re.sub(r'[^\w\s]', '', text).split()
    # words = [word for word in words if not word in stop_words]
    return " ".join([word for word in words])

def cleanAnswer(text):
    text = str(text)
    # wnl = nltk.stem.WordNetLemmatizer()
    text = text.lower()
    words = re.sub(r'[^\w\s]', '', text).split()
    # words = [word for word in words if not word in stop_words]
    return " ".join([word for word in words])

def cleanText(text):
    text = str(text)
    # wnl = nltk.stem.WordNetLemmatizer()
    text = text.lower()
    words = re.sub(r'[^\w\s\.\?]', '', text).split()
    # words = [word for word in words if not word in stop_words]
    return " ".join([word for word in words])

In [7]:
df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))

In [8]:
df.head(1)

ArticleTitle  \
0  Abraham_Lincoln   

                                                           Question Answer  \
0  was abraham lincoln the sixteenth president of the united states  yes     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [9]:
# Preparing data for training
allQuestion = '. '.join(list(df['Question']))
allAnswer = '. '.join(list(df['Answer']))
allContext = '. '.join(list(df['ArticleText']))
text = allQuestion + allAnswer + allContext
s = SplitIndexedList(text.split('.'))

In [10]:
s[0]

(['was',
  'abraham',
  'lincoln',
  'the',
  'sixteenth',
  'president',
  'of',
  'the',
  'united',
  'states'],
 0)

## **Word2Vec**

In [11]:
dataset = []
title = ""
for i in range(0, len(df), 2):
    this_title = df.iloc[i]['ArticleTitle']
    if (this_title!=title):
        title = this_title
        text = df.iloc[i]['ArticleText']
        splitted = text.split(sep='.')
        for j in range(len(splitted)):
            text = splitted[j]
            if(text!=''):
                words = text.split()
                dataset.append(words)
    dataset.append(df.iloc[i]['Question'].split())
    dataset.append(df.iloc[i]['Answer'].split())

In [12]:
model_wv = gensim.models.Word2Vec(dataset, size=100, window=8, min_count=1, sg=0, workers=8) # I have 8 cpu cores
# sg = {0, 1} – Training algorithm: 1 for skip-gram; otherwise CBOW

In [13]:
model_wv.train(dataset, total_examples=len(dataset), compute_loss=True, epochs=50)

(24677141, 31148900)

In [14]:
def get_embedding(sentence):
    pos_sum = [0.0 for i in range(100)]
    num = 0
    words = sentence.split()
    for i in words:
        try:
            embed = model_wv.wv[i]
        except:
            continue
        else:
            pos_sum += embed
            num +=1
    if(num==0):
        return pos_sum
    else:
        pos_sum /= num
        return pos_sum

# get answer using euclidean distance
def get_answer(question, answer_para):
    question_embedding = get_embedding(rem_stop(question))
    min_distance = math.inf
    answer = 0
    for i in range(len(answer_para)):
        answer_embedding = get_embedding(rem_stop(answer_para[i]))
        distance = np.linalg.norm(question_embedding-answer_embedding)
        if (distance < min_distance):
            answer = i
            # print(answer)
            min_distance = distance
    return answer_para[answer]

def rem_stop(sentence):
    strr=''
    my_string = sentence.split()
    for i in range(len(my_string)):
        if my_string[i] not in stopwords.words('english'):
            strr = strr+' '+my_string[i]
    return strr[1:]

# get answer using cosine similarity
def get_answer_cosine(question, answer_para):
    question_embedding = get_embedding(rem_stop(question))
    max_similarity = -math.inf
    answer = 0
    for i in range(len(answer_para)):
        answer_embedding = get_embedding(rem_stop(answer_para[i]))
        similarity = cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(answer_embedding,0))
        if (similarity > max_similarity):
            answer = i
            max_similarity = similarity
    return answer_para[answer]

In [15]:
index = 296
my_text = df.iloc[index]['ArticleText']
temp_sentences = my_text.split(sep='.')
sentences=[]
for i in range(len(temp_sentences)):
    if(temp_sentences[i]!=''):
        sentences.append(temp_sentences[i])
my_question = df.iloc[index]['Question']

# Function to break up article text into individual sentences
def contextToSents(my_text):
    temp_sentences = my_text.split(sep='.')
    sentences=[]
    for i in range(len(temp_sentences)):
        if(temp_sentences[i]!=''):
            sentences.append(temp_sentences[i])
    return sentences

In [16]:
# Word2Vec outputs
print(my_question) # Actual Question
print(rem_stop(my_question)) # Answer without stopwords
print(df.iloc[index]['Answer']) # Actual Answer
print(get_answer(my_question, sentences)) # Our model's prediction using euclidean distance
print("\n")
print(get_answer_cosine(my_question, sentences)) # Our model's prediction using cosine similarity

are there a large number of jews living in egypt today
large number jews living egypt today
no
 the oncevibrant jewish community in egypt has virtually disappeared with only a small number remaining in the country but many egyptian jews visit on religious occasions and for tourism


 the oncevibrant jewish community in egypt has virtually disappeared with only a small number remaining in the country but many egyptian jews visit on religious occasions and for tourism


## **SIF Embedding**

In [17]:
from fse.models import uSIF
glove = api.load("glove-wiki-gigaword-100")
model_sif = uSIF(glove, workers=2, lang_freq="en")

In [18]:
model_sif.train(s)

(1030674, 15765767)

In [19]:
q = 'was abraham lincoln the sixteenth president of the united states'
c1 = 'Abraham Lincoln (February 12, 1809 – April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination.'
c2 = 'Scholars now rank Lincoln among the top three U.S. Presidents, with the majority of those surveyed placing him first.'
c3 = 'Symbolic log cabin at Abraham Lincoln Birthplace National Historic Site.'

tmpq = (q.split(), 0)
tmpc1 = (c1.split(), 0)
tmpc2 = (c2.split(), 0)
tmpc3 = (c3.split(), 0)

print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc1])))
print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc2])))
print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc3])))

[[0.504795]]
[[0.33824894]]
[[0.16190645]]


In [20]:
def getSim(q, x):
    x = (str(x).split(), 0)
    sim = metrics.pairwise.cosine_similarity(model_sif.infer([q]), model_sif.infer([x]))
    return sim

def getAnswer(question, context):
    q = (str(question).split(), 0)
    c = pd.DataFrame(str(context).split('.'))
    c['sim'] = c[0].apply(lambda x: getSim(q, x))
    max = c.sort_values(by='sim', ascending=False).iloc[:3]
    return max

def getBestAnswer(question, potentials):
    q = (str(question).split(), 0)
    c = pd.DataFrame(potentials)
    c['sim'] = c[0].apply(lambda x: getSim(q, x))
    max = c.sort_values(by='sim', ascending=False).iloc[:3]
    return max[0]

In [21]:
qno = 250
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
print(getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno]))

Question:  is foie gras often made from ducks
Top Solutions: 

                                                                                                     0  \
57   foie gras is often made using the liver of domestic ducks rather than of geese                      
53   all domestic ducks are descended from the wild mallard anas platyrhynchos except the muscovy duck   
51   ducks have many economic uses being farmed for their meat eggs feathers particularly their down     

               sim  
57  [[0.90432054]]  
53  [[0.6455733]]   
51  [[0.63837194]]  


In [22]:
qno = 67
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
sol = getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])
print(sol)

Question:  was anders celsius november 27 1701 april 25 1744 a swedish astronomer
Top Solutions: 

                                                                                                                                                                          0  \
4    anders celsius november 27 1701 april 25 1744 was a swedish astronomer                                                                                                   
6    celsius was born in uppsala in sweden                                                                                                                                    
12   celsius founded the uppsala astronomical observatory in 1741 and in 1742 he proposed the celsius temperature scale in a paper to the royal swedish academy of sciences   

               sim  
4   [[0.99999994]]  
6   [[0.6560544]]   
12  [[0.64660877]]  


## **BERT with SIF**

In [23]:
use_cuda = True
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [24]:
def getGraph(start_scores, end_scores, tokens):

    # x-axis: unique tokens
    # y-axis: start word score for each token

    # Pull the scores from Tensors and convert to 1-D np arrays
    start_scores = start_scores.detach().numpy().flatten()
    end_scores = end_scores.detach().numpy().flatten()

    # We add unique token index to each token
    token_label = list()
    for (i, t) in enumerate(tokens):
        token_label.append('{:} - {:>2}'.format(t, i))

    # Plot Graph
    sns.set(style='darkgrid')
    plt.rcParams["figure.figsize"] = (24,8)
    ax = sns.barplot(x=token_labels, y=s_scores, ci=None)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
    ax.grid(True)
    plt.title('Start Word Scores for Each unique Token in Context')
    plt.show()

In [25]:
model_sif.train(SplitIndexedList(list(df['ArticleText'])))

(3420, 14817532)

In [26]:
def get_split(text1):

  #Reference: https://medium.com/@armandj.olivares/using-bert-for-classifying-documents-with-long-texts-5c3e7b04573d
  
    l_total = []
    l_parcial = []
    if len(text1.split())//150 >0:
        n = len(text1.split())//150
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:250]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*150:w*150 + 250]
            l_total.append(" ".join(l_parcial))
    return l_total

In [27]:
def getAnswerBert(question, context):

    # print('Query Context has {} tokens.'.format(len(tokenizer.encode(context))))

    context_list = get_split(context)

    ans = []

    for c in context_list:

        encoding = tokenizer.encode_plus(text=question,text_pair=c)

        inputs = encoding['input_ids']  #Token embeddings
        token_type_id = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

        output = model_bert(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([token_type_id]))
        start_index = torch.argmax(output.start_logits)
        end_index = torch.argmax(output.end_logits)

        answer = ' '.join(tokens[start_index:end_index+1])

        ans.append(answer)
    
    
    print('Question: ', question)

    potentials = []
    for i in ans:
        if ('SEP' not in i) and ('CLS' not in i):
            potentials.append(re.sub('(#)+', '', i))

    answer = getBestAnswer(question, potentials)

    # print('Potential Answers: \n')
    # print(answer.head())
    return answer

In [28]:
print(getAnswerBert(df['Question'].iloc[171], df['ArticleText'].iloc[171]))
print('Actual Answer: ', df['Answer'].iloc[171])

Question:  in 1905 coolidge met and married whom
0    grace anna good hue
1    grace anna good hue
3    grace              
Name: 0, dtype: object
Actual Answer:  grace anna goodhue


In [29]:
print(getAnswerBert(df['Question'].iloc[255], df['ArticleText'].iloc[255]))
print('Actual Answer: ', df['Answer'].iloc[255])

Question:  what kind of ducks feed on land
0    da bbling ducks
1    da bbling      
Name: 0, dtype: object
Actual Answer:  dabbling ducks feed on land


## **Comparision**

In [30]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [31]:
# Function to compare all 3 of our models

def compare(indexes):
    wv_sim = []
    sif_sim = []
    bert_sim = []
    for i in indexes:
        question = df.iloc[i]['Question']
        context = df.iloc[i]['ArticleText']
        answer = df.iloc[i]['Answer']

        if ('yes' in answer.lower()) or ('no' in answer.lower()):
            continue

        ans_sif = model_sif.infer([((answer).split(), 0)])

        w = metrics.pairwise.cosine_similarity(model_sif.infer([((get_answer_cosine(question, contextToSents(context))).split(), 0)]), ans_sif)
        s = metrics.pairwise.cosine_similarity(model_sif.infer([((getAnswer(question, context).iloc[0][0]).split(), 0)]), ans_sif)

        b_ans = getAnswerBert(question, context)
        b = metrics.pairwise.cosine_similarity(model_sif.infer([((b_ans[b_ans.keys()[0]]).split(), 0)]), ans_sif)

        wv_sim.append(w)
        sif_sim.append(s)
        bert_sim.append(b)

    print('WV:', wv_sim)
    print('SIF:', sif_sim)
    print('BERT:', bert_sim)

    return wv_sim, sif_sim, bert_sim

In [32]:
# Function to compare all 3 of our models visually
def visualise(i):

    question = df.iloc[i]['Question']
    context = df.iloc[i]['ArticleText']
    answer = df.iloc[i]['Answer']

    w_ans = get_answer_cosine(question, contextToSents(context))
    s_ans = getAnswer(question, context).iloc[0][0]
    b_ans = getAnswerBert(question, context)

    print('Question: ', question)
    print('Real: ', answer)
    print('WV: ', w_ans)
    print('SIF: ', s_ans)
    print('BERT: ', b_ans[b_ans.keys()[0]])

In [33]:
visualise(130)

Question:  did coolidge graduate from black river academy
Question:  did coolidge graduate from black river academy
Real:  yes
WV:   coolidge graduated from black river academy vermont but failed his initial entrance exam to amherst college
SIF:   coolidge graduated from black river academy vermont but failed his initial entrance exam to amherst college
BERT:  cool idge graduated from black river academy vermont


In [34]:
visualise(234)

Question:  do all ducks quack
Question:  do all ducks quack
Real:  no
WV:   diving ducks and sea ducks forage deep underwater
SIF:   as aforementioned though very few ducks actually do quack
BERT:  most ducks other than female mall ards and domestic ducks do not qu ack


In [35]:
visualise(192)

Question:  how has canada helped un peacekeeping efforts
Question:  how has canada helped un peacekeeping efforts
Real:  during the suez crisis of 1956 lester b pearson eased tensions by proposing the inception of the united nations peacekeeping force canada has since served in 50 peacekeeping missions including every un peacekeeping effort until 1989
WV:   canada has played a leading role in un peacekeeping efforts
SIF:   canada has played a leading role in un peacekeeping efforts
BERT:  canada has played a leading role in un peacekeeping efforts


In [36]:
visualise(777)

Question:  what are four species that are commonly referred to as kangaroos
Question:  what are four species that are commonly referred to as kangaroos
Real:  the red kangaroo the eastern grey kangaroo the western grey kangaroo and the antilopine kangaroo
WV:   there are four species that are commonly referred to as kangaroos
SIF:   there are four species that are commonly referred to as kangaroos
BERT:  the red kangaroo


In [37]:
visualise(666)

Question:  when did james monroe introduce the monroe doctrine
Question:  when did james monroe introduce the monroe doctrine
Real:  december 2 1823
WV:   monroe james
SIF:   monroe james
BERT:  1823


In [38]:
visualise(555)

Question:  why did cleveland want to hide his cancer surgery from the public
Question:  why did cleveland want to hide his cancer surgery from the public
Real:  because of the financial depression of the country
WV:   the public treasury
SIF:   the surgery was conducted through the presidents mouth to avoid any scars or other signs of surgery
BERT:  a cover story about the removal of two bad teeth


In [39]:
visualise(1222)

Question:  what established a trading post on the island in 1819
Question:  what established a trading post on the island in 1819
Real:  british east india company
WV:   the british east india company established a trading post on the island in 1819
SIF:   the british east india company established a trading post on the island in 1819
BERT:  sir thomas stamford raf fles


In [40]:
visualise(342)

Question:  how do elephants communicate over long distances
Question:  how do elephants communicate over long distances
Real:  by producing and receiving lowfrequency sound infrasound
WV:   elephants have a very long childhood
SIF:   it is believed that sound communication between elephants on large distances through the ground is important in their social lives and elephants are observed listening by putting trunks on the ground and carefully moving their very sensitive feet
BERT:  sound communication between elephants on large distances through the ground is important in their social lives and elephants are observed listening by putting trunks on the ground and carefully moving their very sensitive feet


In [41]:
visualise(872)

Question:  what roman province was liechtenstein part of
Question:  what roman province was liechtenstein part of
Real:  raetia
WV:   at one time the territory of liechtenstein formed a part of the ancient roman province of raetia
SIF:   at one time the territory of liechtenstein formed a part of the ancient roman province of raetia
BERT:  holy roman empire


In [42]:
visualise(1242)

Question:  what did roosevelt do to improve his physical condition
Question:  what did roosevelt do to improve his physical condition
Real:  roosevelt took up exercise
WV:   roosevelt timeline
SIF:   to combat his poor physical condition his father compelled the young roosevelt to take up exercise
BERT:  roosevelt for many reasons failed to move enough republicans in his direction


In [46]:
import random
randomlist = random.sample(range(0, len(df)), 10)
output = compare(list(randomlist))
# print(output)
# print(len(output))

Question:  who was the sponsor and mentor of faraday
Question:  which species of zebra is known as the common zebra
Question:  what did alessandro volta invent in 1800
Question:  what is the official language of romania
Question:  oxygen is what
Question:  who founded montevideo
WV: [array([[0.09058279]], dtype=float32), array([[0.8732219]], dtype=float32), array([[0.07141238]], dtype=float32), array([[0.76039886]], dtype=float32), array([[-0.0017149]], dtype=float32), array([[0.4890856]], dtype=float32)]
SIF: [array([[0.60254663]], dtype=float32), array([[0.80794233]], dtype=float32), array([[0.07141238]], dtype=float32), array([[0.6247512]], dtype=float32), array([[0.8876138]], dtype=float32), array([[0.04831631]], dtype=float32)]
BERT: [array([[0.5738723]], dtype=float32), array([[0.7616172]], dtype=float32), array([[0.55611926]], dtype=float32), array([[1.]], dtype=float32), array([[0.39806303]], dtype=float32), array([[0.6212516]], dtype=float32)]


In [47]:
# Final metrics (Average Cosine Similarity Over SIF Embeddings)
print('Word2Vec Avg Similarity: ', sum(output[0])/len(output[0]))
print('SIF Avg Similarity: ', sum(output[1])/len(output[1]))
print('BERTwSIF Avg Similarity: ', sum(output[2])/len(output[2]))

Word2Vec Avg Similarity:  [[0.38049778]]
SIF Avg Similarity:  [[0.5070971]]
BERTwSIF Avg Similarity:  [[0.65182054]]


In [48]:
# Sample user question picked up from the internet along with context
getAnswerBert(('What do they teach at Ashoka?').lower(), ('Ashoka University is a pioneer in its focus on providing a liberal education at par with the best in the world. The aim at Ashoka is to help students become well-rounded individuals who can think critically about issues from multiple perspectives, communicate effectively and become leaders with a commitment to public service. An Ashoka education carries a strong emphasis on foundational knowledge, thorough academic research based on rigorous pedagogy, and hands-on experience with real-world challenging. The 2000-plus students on campus, drawn from 30 states and over 243 cities in India and 27 other countries, receive a world-class interdisciplinary education through undergraduate and post-graduate programmes led by internationally renowned faculty.').lower())

Question:  what do they teach at ashoka?


0    foundation al knowledge , thorough academic research
Name: 0, dtype: object